<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/sec_nlp_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://gist.github.com/Joshua1989/dc7e60aa487430ea704a8cb3f2c5d6a6

# we need to reinitialize local folders and data.

!mkdir sec_data_folder

mkdir: cannot create directory ‘sec_data_folder’: File exists


In [20]:
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
from colab_util import *
drive_handler = GoogleDriveHandler()

Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [0]:
drive_handler.download('implementation_ticker_list.csv', target_path='implementation_ticker_list.csv')

In [14]:
import pandas as pd
import os.path
from os import path

str_id = ''
df_tickers = pd.read_csv('implementation_ticker_list.csv')

for i, row in df_tickers.iterrows():
  str_id = ''
  str_ticker = row['Symbol']
  print('working for:'+str_ticker+"...")
  
  try:
    str_id = drive_handler.path_to_id('/sec_data_folder/'+str_ticker+'.csv')
  except:
    print('/sec_data_folder/'+str_ticker+'.csv not found...')
  
  if len(str_id) > 0:
    str_to_file = 'sec_data_folder/'+str_ticker+'.csv'
    str_from_file = '/sec_data_folder/'+str_ticker+'.csv'
    drive_handler.download(str_to_file, target_path=str_from_file)


working for:MMM...
working for:ABT...
working for:ABBV...
working for:ABMD...
working for:ACN...
working for:ATVI...
working for:ADBE...
working for:AMD...
working for:AAP...
working for:AES...
working for:AFL...
working for:A...
working for:APD...
working for:AKAM...
working for:ALK...
working for:ALB...
working for:ARE...
working for:ALXN...
working for:ALGN...
working for:ALLE...
working for:AGN...
working for:ADS...
working for:LNT...
working for:ALL...
working for:GOOG...
working for:MO...
working for:AMZN...
working for:AMCR...
working for:AEE...
working for:AAL...
working for:AEP...
working for:AXP...
working for:AIG...
working for:AMT...
working for:AWK...
working for:AMP...
working for:ABC...
working for:AME...
working for:AMGN...
working for:APH...
working for:ADI...
working for:ANSS...
working for:ANTM...
working for:AON...
working for:AOS...
working for:APA...
working for:AIV...
working for:AAPL...
working for:AMAT...
working for:APTV...
working for:ADM...
working for:ANET.

In [15]:

#import academic standard NLP library nltk.  this will be our baseline.
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')

import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

#replace with your own list of words ... like covid or delay or cancel (it lowercases automatically)
FIND_WORDS = ['covid',
              'guidance',
              'outlook']

def check_if_list_found_in_text(text, words=[], return_offset=False, lower_text=True):
    result = []
    text = (
        " "
        + text.replace("_", " ")
        .replace("-", " ")
        .replace(",", " ")
        .replace(";", " ")
        .replace('"', " ")
        .replace(":", " ")
        .replace(".", " ")
        + " "
    )
    if lower_text:
        text = text.lower()
    for word in words:
        word = (
            " "
            + word.replace("_", " ")
            .replace("-", " ")
            .replace(",", " ")
            .replace(";", " ")
            .replace('"', " ")
            .replace(":", " ")
            .replace(".", " ")
            + " "
        )
        if lower_text:
            word = word.lower()
        if word in text:
            if return_offset:
                offset = text.find(word)
                # offset = offset if not offset else offset-1
                result.append(offset)
            else:
                result.append(word.strip())
    return result

def filter_stopwords(sent):
    stop_words = set(stopwords.words('english'))   
    word_tokens = word_tokenize(sent)   
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

def df_from_text(text):
    sentence_list = tokenize.sent_tokenize(text)
    sentence_list
    sid = SentimentIntensityAnalyzer()
    list_df = []
    for sentence in sentence_list:
        sentence = filter_stopwords(sentence)
        list_found = check_if_list_found_in_text(sentence,FIND_WORDS)
        num_found = len(list_found)
        ss = sid.polarity_scores(sentence)
        df = pd.DataFrame.from_dict(ss,orient = "index").T
        df['text'] = sentence
        df['keywords_found'] = num_found 
        list_df.append(df)
    return pd.concat(list_df)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [18]:
#read and process data into a single dataset.  add ticker along the way.
import time

master_list_df = []
list_tickers = df_tickers['Symbol']
for ticker in list_tickers:
  
  tic = time.perf_counter()

  import os.path
  from os import path

  if path.exists("sec_data_folder/"+ticker+".csv"):

    df_text = pd.read_csv("sec_data_folder/"+ticker+".csv") 
    df_text['ticker'] = ticker
  
    print("working on..."+ticker)
  
    df_discussion = df_text[df_text['section']=='discussion']

    list_df = []
    for i,row in df_discussion.iterrows(): #this is greedy and should be vectorized with apply once debugged
      df = df_from_text(row['text']) 
      df['id'] = i 
      df['period_date'] = row['period_date']
      df['type'] = row['type']
      df['section'] = row['section']
      list_df.append(df)

    if len(list_df) > 0:
      df_sentence_text_scores = pd.concat(list_df)
      df_sentence_text_scores = df_sentence_text_scores.groupby(['period_date','type']).sum().reset_index()
      df_sentence_text_scores['ticker'] = ticker
      df_sentence_text_scores.drop(['id','keywords_found'],axis = 1)
  
    #scaled scores and differences
      df_sentence_text_scores['compound_baseline'] = df_sentence_text_scores['compound'] / df_sentence_text_scores['compound'].mean()
      df_sentence_text_scores['neg_baseline'] = df_sentence_text_scores['neg'] / df_sentence_text_scores['neg'].mean()
      df_sentence_text_scores['pos_baseline'] = df_sentence_text_scores['pos'] / df_sentence_text_scores['pos'].mean()
      df_sentence_text_scores['compound_bdiff'] = df_sentence_text_scores['compound_baseline'].diff() 
      df_sentence_text_scores['neg_bdiff'] = df_sentence_text_scores['neg_baseline'].diff() 
      df_sentence_text_scores['pos_bdiff'] = df_sentence_text_scores['pos_baseline'].diff() 
      df_sentence_text_scores['compound_zscore'] = (df_sentence_text_scores['compound'] - df_sentence_text_scores['compound'].mean())/df['compound'].std(ddof=0)

      master_list_df.append(df_sentence_text_scores)
    else:
      print("missing..."+ticker)
    toc = time.perf_counter()
    print(f"Text processed in {toc - tic:0.4f} seconds")


df_data = pd.concat(master_list_df)


working on...MMM
Text processed in 47.9999 seconds
working on...ABT
Text processed in 16.6962 seconds
working on...ABBV
Text processed in 25.5357 seconds
working on...ABMD
Text processed in 23.0224 seconds
working on...ACN
Text processed in 35.5953 seconds
working on...ATVI
Text processed in 44.8626 seconds
working on...ADBE
Text processed in 39.5123 seconds
working on...AMD
Text processed in 25.3486 seconds
working on...AAP
Text processed in 21.4858 seconds
working on...AES
Text processed in 74.8815 seconds
working on...AFL
Text processed in 82.6329 seconds
working on...A
Text processed in 39.2829 seconds
working on...APD
Text processed in 55.8449 seconds
working on...AKAM
Text processed in 38.1558 seconds
working on...ALK
Text processed in 45.7255 seconds
working on...ALB
Text processed in 50.6044 seconds
working on...ARE
Text processed in 109.3433 seconds
working on...ALXN
Text processed in 56.8385 seconds
working on...ALGN
Text processed in 35.8298 seconds
working on...ALLE
Text pr

In [21]:
df_data.to_csv("df_data.csv")
drive_handler.upload('df_data.csv', parent_path='sec_data_folder',overwrite=False)

'169khhxPuGChoX1n0teyJs7zrULWx3pBj'

In [27]:
df = df_data[['period_date','ticker','compound_baseline']]
df['quarter_end'] = pd.to_datetime(df['period_date'])
df['quarter_end'] = df['quarter_end'].dt.to_period('q').dt.end_time
df['quarter_end'] = df['quarter_end'].dt.date
import numpy as np
df_data_pivot = pd.pivot_table(df, values='compound_baseline', index=['quarter_end'],
                    columns=['ticker'], aggfunc=np.sum, fill_value=0).reset_index()

df_data_pivot.to_csv("df_data_pivot.csv")
drive_handler.upload('df_data_pivot.csv', parent_path='sec_data_folder',overwrite=False)
print('done!')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1104: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/di

done!


In [28]:
df_data_pivot

ticker,quarter_end,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,...,EFX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,FANG,FAST,FB,FDX,FE,FFIV,FIS,FITB,FRT,GLW,GOOG,KMX,KO,LNT,MMM,MO,RE,SCHW,STZ,T,XOM,XRAY
0,2017-03-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.957626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.711627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2017-06-30,0.873429,1.104600,0.732465,0.000000,0.703215,1.018923,1.022215,0.756768,1.132543,0.920287,1.029541,0.995754,1.241189,1.115083,0.900043,0.852770,0,1.415951,0.856532,1.210443,1.288804,0.918404,0.613515,0.978009,0.841702,0.824256,0.786310,1.069966,0.000000,0.821801,0.682505,0.896560,0.000000,0.000000,1.022294,0.640022,1.028938,0.947435,0.954852,...,0.732661,1.579053,0.622491,0.000000,0.927942,0.000000,1.148993,0.000000,0.774241,0.955044,0.986450,0,0.000000,0.876581,1.270582,0.964980,0.849069,1.363261,0.783024,0.853841,0.975714,0.000000,0.909149,0.955683,0.848285,0,1.077555,0.828397,0.815994,0.000000,0.991665,0.868103,0.946485,1.023027,1.280445,0.804079,0.699945,1.353261,1.397320,1.089609
2,2017-09-30,0.864391,1.023225,0.789582,2.584671,0.767476,1.559986,1.065875,0.749377,1.289192,0.891824,1.092399,0.764374,0.717611,1.071293,0.877645,0.972885,0,1.320607,0.890857,1.344186,1.339411,0.940734,0.563404,0.989094,0.866690,0.876355,0.822627,1.076938,0.000000,0.932362,0.818635,0.863265,0.000000,4.241852,1.087055,0.732248,1.134848,0.959780,0.931219,...,1.065335,0.683411,0.643215,0.922393,0.944634,0.000000,0.973895,0.000000,0.812737,0.935044,1.617602,0,0.000000,0.963036,1.249455,1.075268,0.867162,1.374646,0.796563,0.770770,0.980335,1.217621,0.994614,1.997653,0.785867,0,1.253129,0.888433,0.830640,0.000000,0.918472,0.980205,0.976129,0.957941,0.893738,0.808683,0.938340,1.377402,1.669807,1.171936
3,2017-12-31,1.499952,1.648941,2.134255,1.259892,1.787875,0.784884,1.059573,1.994179,0.734767,1.428478,1.729166,1.373037,0.929142,1.682805,0.921097,1.394077,0,1.743915,1.142466,2.346148,1.021130,1.240867,1.444324,1.294904,1.352717,1.611396,1.495689,1.914996,0.970234,1.599878,1.388205,1.252196,0.000000,3.178048,0.000000,2.054919,1.287514,1.335258,1.193343,...,1.670499,0.817254,1.634602,0.000000,1.276090,1.322674,1.357549,1.052718,1.354526,0.981600,0.000000,1,0.000000,1.740006,1.810543,1.112201,1.586140,1.784867,1.239695,1.495806,1.465482,1.563253,1.360784,0.537943,1.554680,0,1.507679,2.301632,1.375743,0.000000,1.624128,2.953641,1.394643,1.108436,0.000000,1.483551,1.105233,0.000000,0.014181,1.346699
4,2018-03-31,0.777040,0.832421,0.000000,1.425279,0.740934,0.890735,1.051170,0.686984,1.022810,0.837214,0.806753,0.594576,0.986419,0.546166,1.304820,0.664638,0,0.745252,0.857155,0.498048,1.402148,0.893626,0.779970,0.869974,0.789987,0.705634,0.796147,0.741784,0.000000,0.588661,0.705277,0.926894,0.000000,0.502837,0.976424,0.597081,0.738582,0.934052,0.927822,...,0.653581,0.900389,0.895870,0.000000,0.733878,0.000000,0.875269,0.000000,0.680906,0.870707,1.429944,0,0.000000,0.686395,0.978899,0.888084,0.869493,0.610593,0.716393,0.675056,0.885016,1.511688,0.890781,0.752764,0.809118,0,0.636957,0.546437,0.786791,1.032003,0.834326,0.595038,0.760088,0.982709,1.145317,0.643665,1.373749,1.261949,1.108457,0.716795
5,2018-06-30,0.910148,1.104417,0.816777,1.401048,0.786992,0.898001,0.990159,0.729177,1.176514,0.855999,0.941293,1

In [0]:
df_data.head(6)

In [0]:
df_grouped = df_data[df_data['ticker'] == 'GOOG']
df_grouped['period_date'] = pd.to_datetime(df_grouped['period_date'])
df_grouped['period_date'] = df_grouped['period_date'].dt.strftime('%m/%d/%Y')
df_grouped.plot(x='period_date',y='compound_baseline',kind = 'bar',title = 'GOOG Baseline MD&A Sentiment NLTK')

#to do:
#debug why 6 periods
#delta baseline vs following 5 day alpha in stock
#ARIMA model fit (or sum of squares for easy explanation).
#the NLTK implementation is the base standard of comparison

#pytorch transformers sentiment analysis - language model
#as above, delta baseline vs alpha
#compare w NLTK ARIMA/linear regression fit.
